## Load Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils

from functools import partial as func_partial
from functools import reduce as func_reduce
from operator import mul as op_mul
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import PopulationBasedTraining
from os import cpu_count, path
from time import strftime

#This class contains DatasetClass and several helper functions
import ClassModule as cm

In [2]:
# Show number of avlaible CPU threads
# With mulithreading this number is twice the number of physical cores
cpu_av = cpu_count()
print("Number of available CPU's: {}".format(cpu_av))

Number of available CPU's: 12


In [3]:
# Set the number CPUS that should be used per trial and dataloader
# If set to 1 number of cucurrent training networking is equal to this number
# In case of training with GPU this will be limited to number of models training simultaneously on GPU
# So number of CPU threads for each trial can be increased 
cpus_per_trial = 1
gpus_per_trial = 0

In [4]:
def get_dataloader(train_ds, val_ds, bs):
    dl_train = utils.DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=cpus_per_trial-1)
    dl_val = utils.DataLoader(val_ds, batch_size=bs * 2, shuffle=True, num_workers=cpus_per_trial-1)
    return  dl_train, dl_val

## Instance Noise

In [5]:
# https://arxiv.org/abs/1610.04490
INSTANCE_NOISE = False

def add_instance_noise(data, std=0.01):
    return data + torch.distributions.Normal(0, std).sample(data.shape).to(device)

## Define the network
Get a network-candidate from the ASHA-scheduler first and use this notebook for hyperparameter tuning

In [6]:
class CNN(nn.Module):
    def __init__(self, input_dim=(2,20,20), num_in_features=5):
        super(CNN, self).__init__()
        self.feature_ext = nn.Sequential(
            nn.Conv2d(2,10, kernel_size=3, padding=2),
            nn.ReLU(),
            nn.Conv2d(10,10, kernel_size=3,  padding=2),
            nn.ReLU(),
            nn.Conv2d(10,10, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(10,5, kernel_size=1, padding=0),
            nn.ReLU()
        )
        
        self.flatten = nn.Flatten()
        
        # Gives the number of features after the conv layer
        num_features_after_conv = func_reduce(op_mul, list(self.feature_ext(torch.rand(1, *input_dim)).shape))
        
        self.dense_nn = nn.Sequential(
            nn.Linear(num_features_after_conv + num_in_features, 4),
            nn.ReLU(),
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 3),
            nn.ReLU()
        )
        
    def forward(self, cluster, clusNumXYEPt):
        cluster = self.feature_ext(cluster)
        x = self.flatten(cluster)
        x = torch.cat([x, clusNumXYEPt], dim=1)
        logits = self.dense_nn(x)
        return logits

## Implement train and validation loop
[0: 'ClusterN', 1:'Cluster', 2:'ClusterTiming', 3:'ClusterType', 4:'ClusterE', 5:'ClusterPt', 6:'ClusterModuleNumber', 7:'ClusterRow', 8:'ClusterCol', 9:'ClusterM02', 10:'ClusterM20', 11:'ClusterDistFromVert', 12:'PartE', 13:'PartPt', 14:'PartEta', 15:'PartPhi', 16:'PartIsPrimary', 17:'PartPID']

In [7]:
def train_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):

    size = len(dataloader.dataset)
    running_loss = 0.0
    epoch_steps = 0

    for batch, Data in enumerate(dataloader):
        Clusters = Data[0].to(device)
        Features = cm.unsqueeze_features(Data[1])
        Labels = Data[2]
        
        ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1).to(device)
        #Labels = torch.cat([Labels["PartPID"], dim=1]).to(device)
        Label = Labels["PartPID"].to(device)
        
        if INSTANCE_NOISE:
            Clusters = add_instance_noise(Clusters, device)
        
        # zero parameter gradients
        optimizer.zero_grad()
        
        # prediction and loss
        pred = model(Clusters, ClusterProperties)
        loss = loss_fn(pred, Label.long())
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        epoch_steps += 1
        
        if batch % 100000 == 99999:
            print("[%d, %5d] loss: %.3f" % (epoch + 1, batch + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0               

In [8]:
def val_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):
    
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    size = len(dataloader.dataset)
    
    for batch, Data in enumerate(dataloader):
        with torch.no_grad():
            Clusters = Data[0].to(device)
            Features = cm.unsqueeze_features(Data[1])
            Labels = Data[2]
            ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1).to(device)           
            #Labels = torch.cat([Labels["PartPID"], dim=1]).to(device)
            Label = Labels["PartPID"].to(device)
            
            pred = model(Clusters, ClusterProperties)
            correct += (pred.argmax(1) == Label).type(torch.float).sum().item()

            loss = loss_fn(pred, Label.long())#.item()
            val_loss += loss.cpu().numpy()
            val_steps += 1
    
    with tune.checkpoint_dir(epoch) as checkpoint_dir:
        _path = path.join(checkpoint_dir, "checkpoint")
        torch.save((model.state_dict(), optimizer.state_dict()), _path)
        
    tune.report(loss=(val_loss / val_steps), accuracy= correct / size)

## Implement method for accuracy testing on test set

In [9]:
def test_accuracy(model, device="cpu"):
    
    dataset_test = cm.load_data_test('/media/DATA/ML-Notebooks/CNN/Data/data_test.npz')
    
    dataloader_test = utils.DataLoader(
        dataset_test, batch_size=4, shuffle=False, num_workers=cpu_av-1)
    
    correct = 0
    total = len(dataloader_test.dataset)
    
    with torch.no_grad():
        for batch, Data in enumerate(dataloader_test):
            Clusters = Data[0].to(device)
            Features = cm.unsqueeze_features(Data[1])
            Labels = Data[2]
            ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1).to(device)            
            #Labels = torch.cat([Labels["PartPID"], dim=1]).to(device)
            Label = Labels["PartPID"].to(device)
            
            
            pred = model(Clusters, ClusterProperties)
            correct += (pred.argmax(1) == Label).type(torch.float).sum().item()

    return correct / total

## Implement training routine

In [13]:
def train_model(config, checkpoint_dir=None):
    
    # load model
    model = CNN()
    
    # check for avlaible resource and initialize device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
    # send model to device
    model.to(device)
    
    # initialise loss function and opptimizer
    loss_fn = F.cross_entropy
    optimizer = torch.optim.Adam(model.parameters(),lr=config["lr"], weight_decay=config["wd"])
    
    # check whether checkpoint is available
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    # load dataset
    dataset_train = cm.load_data_train()
    
    # split trainset in train and validation subsets
    test_abs = int(len(dataset_train) * 0.8)
    subset_train, subset_val = utils.random_split(
        dataset_train, [test_abs, len(dataset_train) - test_abs])

    # get dataloaders 
    dataloader_train, dataloader_val = get_dataloader(subset_train, subset_val, int(config["batch_size"]))
                                                      
    for epoch in range(100):
        train_loop(epoch, dataloader_train, model, loss_fn, optimizer, device=device)
        val_loop(epoch, dataloader_train, model, loss_fn, optimizer, device=device)                                              
    
    print("Finished Training")

## Setup all Ray Tune functionality and start training

In [17]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    
    # Setup hyperparameter-space to search
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "wd" : tune.uniform(0, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16, 32, 64])
    }

    # Init the scheduler
    scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="accuracy",
    mode="max",
    perturbation_interval=10)
        
    
    # Init the Reporter
    reporter = CLIReporter(
        parameter_columns=["lr", "wd", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    #Get Current date and time
    timestr = strftime("%Y_%m_%d-%H:%M:%S")
    name = "ASHA-" + timestr
    
    # Init the run method
    result = tune.run(
        func_partial(train_model),
        name = name,
        resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        local_dir = "./Ray_Results",
        scheduler=scheduler,
        progress_reporter=reporter)
    
    # Find best trial and use it on the testset
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))
    
    best_trained_model = CNN()
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)
    
    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)
    
    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))

In [18]:
main(num_samples=10, max_num_epochs=100, gpus_per_trial=gpus_per_trial)

2021-09-22 15:37:23,728	WARNING experiment.py:295 -- No name detected on trainable. Using DEFAULT.
2021-09-22 15:37:23,729	INFO registry.py:66 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 6.2/15.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 1.0/12 CPUs, 0/0 GPUs, 0.0/5.87 GiB heap, 0.0/2.93 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_22-15:37:23
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+-------------+------------+--------------+
| Trial name          | status   | loc   |          lr |         wd |   batch_size |
|---------------------+----------+-------+-------------+------------+--------------|
| DEFAULT_37dfb_00000 | RUNNING  |       | 0.000190831 | 0.00746175 |           64 |
| DEFAULT_37dfb_00001 | PENDING  |       | 0.0734565   | 0.0429869  |            8 |
| DEFAULT_37dfb_00002 | PENDING  |       | 0.0893902   | 0.0772182  |           32 |
| DEFAULT_37dfb_00003 | PENDING  |       | 0.00804634  | 0.0162324  |            4 |
| DEFAULT_37dfb_00004 | PENDING  |       | 0.0132987   | 0.0968306  |           16 |
|

(pid=26201) [3,    10] loss: 0.910
(pid=26203) [4,    10] loss: 0.808
(pid=26211) [2,    20] loss: 0.549
(pid=26211) [2,    30] loss: 0.366
(pid=26207) [3,    10] loss: 0.811
(pid=26201) [3,    20] loss: 0.386
(pid=26201) [3,    30] loss: 0.263
(pid=26211) [2,    40] loss: 0.275
(pid=26211) [2,    50] loss: 0.220
(pid=26205) [3,    10] loss: 0.848
(pid=25556) [6,    10] loss: 0.830
(pid=26205) [3,    20] loss: 0.353
(pid=26211) [2,    60] loss: 0.183
(pid=26211) [2,    70] loss: 0.157
(pid=26203) [5,    10] loss: 0.766
(pid=26205) [3,    30] loss: 0.310
(pid=26208) [4,    10] loss: 0.824
(pid=26207) [4,    10] loss: 0.808
(pid=26201) [4,    10] loss: 0.947
(pid=25556) [7,    10] loss: 0.798
(pid=26201) [4,    20] loss: 0.460
(pid=26201) [4,    30] loss: 0.256
(pid=26203) [6,    10] loss: 0.772
(pid=26205) [4,    10] loss: 0.811
(pid=26211) [3,    10] loss: 1.099
(pid=26205) [4,    20] loss: 0.358
(pid=26208) [5,    10] loss: 0.857
(pid=26211) [3,    20] loss: 0.549
(pid=26207) [5,    1

2021-09-22 15:37:28,400	INFO pbt.py:540 -- [exploit] transferring weights from trial DEFAULT_37dfb_00002 (score 0.7142857142857143) -> DEFAULT_37dfb_00001 (score 0.7006802721088435)
2021-09-22 15:37:28,404	INFO pbt.py:557 -- [explore] perturbed config from {} -> {}


(pid=26203) [10,    10] loss: 0.768
(pid=26205) [6,    30] loss: 0.311
(pid=26211) [4,    60] loss: 0.143
(pid=26211) [4,    70] loss: 0.106
(pid=26201) [7,    10] loss: 0.876
(pid=26207) [8,    10] loss: 0.804
(pid=25556) [10,    10] loss: 0.856
(pid=26201) [7,    20] loss: 0.355
(pid=26201) [7,    30] loss: 0.339
(pid=26205) [7,    10] loss: 0.776
(pid=26211) [5,    10] loss: 0.754


...

(pid=26198) [10,    10] loss: 0.901
(pid=26211) [99,    10] loss: 0.746
(pid=26198) [1,    10] loss: 0.819
(pid=26211) [100,    10] loss: 0.752
(pid=26198) [2,    10] loss: 0.754
Result for DEFAULT_37dfb_00008:
  accuracy: 0.7278911564625851
  date: 2021-09-22_15-39-19
  done: true
  experiment_id: 82f82040a1e2446cba0aacaeb16ff210
  experiment_tag: 8_batch_size=2,lr=0.0022545,wd=0.049953@perturbed[]
  hostname: jhonerma-tuxedo
  iterations_since_restore: 100
  loss: 0.733969646692276
  node_ip: 10.67.95.37
  pid: 26211
  should_checkpoint: true
  time_since_restore: 17.37765121459961
  time_this_iter_s: 0.16582465171813965
  time_total_s: 99.9757788181305
  timestamp: 1632317959
  timesteps_since_restore: 0
  training_iteration: 470
  trial_id: 37dfb_00008
  
(pid=26211) Finished Training
(pid=26198) [3,    10] loss: 0.792
(pid=26198) [4,    10] loss: 0.789
== Status ==
Memory usage on this node: 6.3/15.1 GiB
PopulationBasedTraining: 89 checkpoints, 71 perturbs
Resources requested: 1.0

(pid=26198) [47,    10] loss: 0.805
(pid=26198) [48,    10] loss: 0.774
(pid=26198) [49,    10] loss: 0.787
(pid=26198) [50,    10] loss: 0.819
(pid=26198) [51,    10] loss: 0.762
(pid=26198) [52,    10] loss: 0.792
(pid=26198) [53,    10] loss: 0.795
(pid=26198) [54,    10] loss: 0.753
(pid=26198) [55,    10] loss: 0.765
Result for DEFAULT_37dfb_00002:
  accuracy: 0.7006802721088435
  date: 2021-09-22_15-39-28
  done: false
  experiment_id: 82f82040a1e2446cba0aacaeb16ff210
  hostname: jhonerma-tuxedo
  iterations_since_restore: 56
  loss: 0.786327201128006
  node_ip: 10.67.95.37
  pid: 26198
  should_checkpoint: true
  time_since_restore: 9.25491189956665
  time_this_iter_s: 0.1618356704711914
  time_total_s: 107.51363587379456
  timestamp: 1632317968
  timesteps_since_restore: 0
  training_iteration: 516
  trial_id: 37dfb_00002
  
(pid=26198) [56,    10] loss: 0.823
(pid=26198) [57,    10] loss: 0.793
(pid=26198) [58,    10] loss: 0.790
(pid=26198) [59,    10] loss: 0.793
(pid=26198)

(pid=26198) [98,    10] loss: 0.765
(pid=26198) [99,    10] loss: 0.803


2021-09-22 15:39:36,467	INFO tune.py:561 -- Total run time: 132.74 seconds (132.56 seconds for the tuning loop).


Result for DEFAULT_37dfb_00002:
  accuracy: 0.7006802721088435
  date: 2021-09-22_15-39-36
  done: true
  experiment_id: 82f82040a1e2446cba0aacaeb16ff210
  experiment_tag: 2_batch_size=32,lr=0.08939,wd=0.077218@perturbed[]
  hostname: jhonerma-tuxedo
  iterations_since_restore: 100
  loss: 0.7595935583114624
  node_ip: 10.67.95.37
  pid: 26198
  should_checkpoint: true
  time_since_restore: 16.613488912582397
  time_this_iter_s: 0.15553045272827148
  time_total_s: 114.8722128868103
  timestamp: 1632317976
  timesteps_since_restore: 0
  training_iteration: 560
  trial_id: 37dfb_00002
  
(pid=26198) [100,    10] loss: 0.768
(pid=26198) Finished Training
== Status ==
Memory usage on this node: 6.2/15.1 GiB
PopulationBasedTraining: 89 checkpoints, 71 perturbs
Resources requested: 0/12 CPUs, 0/0 GPUs, 0.0/5.87 GiB heap, 0.0/2.93 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_22-15:37:23
Number of trials: 10/10 (10 TERMINATED)
+---------------------+-----